In [1]:
import numpy as np
import pandas as pd
import syft as sy
import time
import os
import dotenv
import sqlite3


In [4]:
# we recommend that you source these values using environment variables
dotenv.load_dotenv()
URL = "https://reddit-for-researchers.snooguts.net"
EMAIL = os.getenv("EMAIL")
PASSWORD = os.getenv("PASSWORD")

# you can provide a "password" keyword argument, but if you don't...
# the browser will prompt you for input
client = sy.login(
    url=URL,
    email=EMAIL,
    password=PASSWORD
)

Logged into <reddit-for-researchers-internal: High side Datasite> as <prgildersleve@gmail.com>


In [ ]:
client.requests

In [7]:
def name_to_id_dict(client):
    # TODO handle case with duplicate names?
    return {request.code.service_func_name: str(request.id) for request in client.requests}


def query_r4r(function_name, name_to_id_dict, client):

    while True:   
        try:
            request = client.api.services.request.get_by_uid(uid=sy.UID(name_to_id_dict[function_name]))
            job = request.code(blocking=False)
            results = job.wait()
            df = results.get()
            return df
        except KeyboardInterrupt:
            raise
        except Exception as ex:
            print(ex)
            time.sleep(10)
            client = sy.login(
                url=URL,
                email=EMAIL,
                password=PASSWORD
            )

In [ ]:
funcdict = name_to_id_dict(client)
funcdict

In [ ]:
request = client.api.services.request.get_by_uid(uid=sy.UID(funcdict["avg_comment_score_pg"]))
job = request.code(blocking=False)
results = job.wait()
results.get()

In [ ]:
# Posts

posts_dict = {}
for year in range(2020, 2024):
    key = f"pg_wiki_{year}"
    print(key)
    try:
        posts_dict[key] = pd.read_hdf(f"data/{key}.h5")
    except:
        posts_dict[key] = query_r4r(key, funcdict, client)
        for c in ['score', 'gildings', 'num_comments']:
            posts_dict[key][c] = posts_dict[key][c].astype(np.int64)
        for c in ['nsfw', 'self', 'video', 'locked', 'spoiler', 'sticky']:
            posts_dict[key][c] = posts_dict[key][c].astype(np.bool_)
        for c in ['created_at', 'updated_at']:
            posts_dict[key][c] = pd.to_datetime(posts_dict[key][c]).astype('datetime64[ns]')

        posts_dict[key].to_hdf(f"data/{key}.h5", key='df', mode='w')

posts_df = pd.concat(posts_dict.values())
posts_df.to_hdf("data/posts.h5", key='df', mode='w')


In [2]:
posts_df = pd.read_hdf("data/posts.h5")


In [ ]:
# Comments

comments_dict = {}
for year in range(2020, 2024):
    for month in range(1, 13):
        key = f"pg_wiki_comments_{year}{month:02d}"
        print(key)
        try:
            comments_dict[key] = pd.read_hdf(f"data/{key}.h5")
        except:
            comments_dict[key] = query_r4r(key, funcdict, client)
            for c in ['score']:
                comments_dict[key][c] = comments_dict[key][c].astype(np.int64)
            for c in ['gilded']:
                comments_dict[key][c] = comments_dict[key][c].astype(np.bool_)
            for c in ['created_at', 'last_modified_at']:
                comments_dict[key][c] = pd.to_datetime(comments_dict[key][c]).astype('datetime64[ns]')
            comments_dict[key].to_hdf(f"data/{key}.h5", key='df', mode='w')

comments_df = pd.concat(comments_dict.values())
os.remove("data/comments_1.h5")
comments_df.iloc[:len(comments_df)//4].to_hdf("data/comments_1.h5", key='df', mode='w')
os.remove("data/comments_2.h5")
comments_df.iloc[len(comments_df)//4:len(comments_df)//2].to_hdf("data/comments_2.h5", key='df', mode='w')
os.remove("data/comments_3.h5")
comments_df.iloc[len(comments_df)//2:len(comments_df)//4*3].to_hdf("data/comments_3.h5", key='df', mode='w')
os.remove("data/comments_4.h5")
comments_df.iloc[len(comments_df)//4*3:].to_hdf("data/comments_4.h5", key='df', mode='w')

In [ ]:
#  replies

replies_df = query_r4r('pg_wiki_replies', funcdict, client)
for c in ['score']:
    replies_df[c] = replies_df[c].astype(np.int64)
for c in ['gilded']:
    replies_df[c] = replies_df[c].astype(np.bool_)
for c in ['created_at', 'last_modified_at']:
    replies_df[c] = pd.to_datetime(replies_df[c]).astype('datetime64[ns]')
replies_df.to_hdf(f"data/replies.h5", key='df', mode='w')

In [ ]:
# replies

replies_dict = {}
for year in range(2020, 2024):
    key = f"pg_wiki_replies_{year}"
    print(key)
    try:
        replies_dict[key] = pd.read_hdf(f"data/{key}.h5")
    except:
        replies_dict[key] = query_r4r(key, funcdict, client)
        for c in ['score']:
            replies_dict[key][c] = replies_dict[key][c].astype(np.int64)
        for c in ['gilded']:
            replies_dict[key][c] = replies_dict[key][c].astype(np.bool_)
        for c in ['created_at', 'last_modified_at']:
            replies_dict[key][c] = pd.to_datetime(replies_dict[key][c]).astype('datetime64[ns]')

        replies_dict[key].to_hdf(f"data/{key}.h5", key='df', mode='w')

replies_df = pd.concat(replies_dict.values())
os.remove("data/replies.h5")
replies_df.to_hdf("data/replies.h5", key='df', mode='w')

In [ ]:
replies_dict = {}
for year in range(2020, 2024):
    for month in range(1, 13):
        key = f"pg_wiki_replies_{year}"
        print(key)
        try:
            replies_dict[key] = pd.read_hdf(f"data/{key}.h5")
        except:
            replies_dict[key] = query_r4r(key, funcdict, client)
            for c in ['score']:
                replies_dict[key][c] = replies_dict[key][c].astype(np.int64)
            for c in ['gilded']:
                replies_dict[key][c] = replies_dict[key][c].astype(np.bool_)
            for c in ['created_at', 'last_modified_at']:
                replies_dict[key][c] = pd.to_datetime(replies_dict[key][c]).astype('datetime64[ns]')

            replies_dict[key].to_hdf(f"data/{key}.h5", key='df', mode='w')

replies_df = pd.concat(replies_dict.values())
os.remove("data/replies.h5")
replies_df.to_hdf("data/replies.h5", key='df', mode='w')

In [5]:
posts_df = pd.read_hdf("data/posts.h5")
comments_df = pd.concat([pd.read_hdf(f"data/comments_{i}.h5") for i in range(1, 5)])

In [8]:
posts_df.loc[posts_df['updated_at'] == pd.Timestamp('1970-01-01 00:00:00'), 'updated_at'] = pd.NaT
comments_df.loc[comments_df['last_modified_at'] == pd.Timestamp('1970-01-01 00:00:00'), 'last_modified_at'] = pd.NaT

posts_df.to_hdf("data/posts.h5", key='df', mode='w')
for i in range(1, 5):
    os.remove(f"data/comments_{i}.h5")
    comments_df.iloc[len(comments_df)//4*(i-1):len(comments_df)//4*i].to_hdf(f"data/comments_{i}.h5", key='df', mode='w')

/tmp/ipykernel_132352/3707721486.py:4: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'subreddit_id', 'title', 'body', 'url', 'author_id',
       'distinguished', 'flair_text', 'language_code', 'thumbnail',
       'crosspost_parent_id', 'permalink'],
      dtype='object')]

  posts_df.to_hdf("data/posts.h5", key='df', mode='w')
/tmp/ipykernel_132352/3707721486.py:7: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['id', 'post_id', 'parent_id', 'body', 'author_id', 'subreddit_id',
       'permalink'],
      dtype='object')]

  comments_df.iloc[len(comments_df)//4*(i-1):len(comments_df)//4*i].to_hdf(f"data/comments_{i}.h5", key='df', mode='w')
/tmp/ipykernel_132352/3707721486.py:7: PerformanceWarning: 
your performance

In [4]:
display(posts_df['updated_at'][(posts_df['updated_at'] == pd.Timestamp('1970-01-01 00:00:00'))])
display(comments_df['last_modified_at'][(comments_df['last_modified_at'] == pd.Timestamp('1970-01-01 00:00:00'))])

Series([], Name: updated_at, dtype: datetime64[ns])

Series([], Name: last_modified_at, dtype: datetime64[ns])

In [5]:
posts_df.loc[posts_df['updated_at'] == pd.Timestamp('1970-01-01 00:00:00'), 'updated_at'] = pd.NaT
comments_df.loc[comments_df['last_modified_at'] == pd.Timestamp('1970-01-01 00:00:00'), 'last_modified_at'] = pd.NaT
ccols = ['subreddit_id', 'post_id', 'parent_id', 'id', 'created_at',
         'last_modified_at', 'score', 'upvote_ratio', 'gilded']
cw = comments_df[ccols]
pcols = ['subreddit_id', 'crosspost_parent_id', 'id', 'created_at', 'updated_at',
         'language_code', 'score', 'upvote_ratio', 'gildings', 'num_comments']
pw = posts_df[pcols]

In [11]:
#  create sqlite database
conn = sqlite3.connect('wikireddit.db')
posts_df[pcols].to_sql('posts', conn, index=False, if_exists='replace')
comments_df[ccols].to_sql('comments', conn, index=False, if_exists='replace')

10264340

In [ ]:
# test read some rows from each table
conn = sqlite3.connect('wikireddit.db')
display(pd.read_sql('SELECT * FROM posts LIMIT 5', conn))
display(pd.read_sql('SELECT * FROM comments LIMIT 5', conn))

In [13]:
# get storage size of database
import os
mb_size = os.path.getsize('wikireddit.db') / 1024 / 1024
print(f"Database size: {mb_size:.2f} MB")

Database size: 1191.16 MB
